In [1]:
from aesim.simba import  ProjectRepository, License
import matplotlib.pyplot as plt
import numpy as np
import os
import time as t
from scipy.optimize import Bounds, NonlinearConstraint, differential_evolution, minimize
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
number_of_parallel_simulations = License.NumberOfAvailableParallelSimulationLicense() # Number of available parallel simulation license

#  Using scipy to optimize PID parameters of a simple Buck Converter with Voltage Regulation
![Buck-Converter](fig/buck.png)

Optimization objectives are:

* The output voltage should be equal to the reference voltage (error = 0, non-linear constraint)
* No overshoot (overshoot = 0, non-linear constraint)
* Minimize risetime


In [2]:
# Open design and get the pid device
file_path = os.path.join(os.getcwd(), "Buck.jsimba")
project = ProjectRepository(file_path)
buck = project.GetDesignByName('DC/DC - Buck Converter')
pid = buck.Circuit.GetDeviceByName('PID1')

In [3]:
# simulation_history holds the parameters (Ki, Kp and Kd) and the metrics (risetime, error and overshoot) previously calculated
simulation_history = {}
def create_key(X):
    return str(X[0]) + str(X[1]) + str(X[2])

#  Rise time is defined as the time to reach 95% of the final value
def rise_time(v,t):
    v_flip = np.flip(v)
    t_flip = np.flip(t)
    v_last = v_flip[0]
    v_first = v_flip[-1]
    v_95 = 0.95*(v_last-v_first)
    index = np.where(v_flip < v_95)[0][0]
    return t_flip[index]

# Run the simulation and update the simulation_history dictionary
def evaluate(X):
    # Apply parameters
    pid.Ki= float(X[0])
    pid.Kp = float(X[1])
    pid.Kd= float(X[2])

    # Run simulation
    job = buck.TransientAnalysis.NewJob()
    status = job.Run()
    if str(status) != "OK": 
        raise Exception(job.Summary())
    
    # Calculate metrics and update simulation_history
    Vout_signal = job.GetSignalByName('R1 - Instantaneous Voltage')
    t =  np.array(Vout_signal.TimePoints)
    Vout = np.array(Vout_signal.DataPoints)
    risetime = rise_time(Vout,t) 
    v_ref = 4
    v_last = Vout[-1]
    error = abs(v_ref-v_last)/v_ref
    overshoot = max(Vout)/v_last-1
    key = create_key(X)
    simulation_history[key] = [risetime, error, overshoot]

In [4]:
def objective3D(X):
   # calculate and return the rise time (objective to minimize)
    key = create_key(X)
    if not key in simulation_history.keys():
        evaluate(X)
    return simulation_history.get(key)[0]

def constraintError3D(X):
   # Calculate and return the output voltage error (non linear constraint)
    key = create_key(X)
    if not key in simulation_history.keys():
        evaluate(X)
    return simulation_history.get(key)[1]

def constraintOverShoot3D(X):
    # Calculate and return the overshoot (non linear constraint)
    key = create_key(X)
    if not key in simulation_history.keys():
        evaluate(X)
    return simulation_history.get(key)[2]

In [5]:
# Define nonlinear constraints (no error and no overshoot)
nonlinear_constraint_err = NonlinearConstraint(constraintError3D, 0.0, 10**(-2))
nonlinear_constraint_OverShoot = NonlinearConstraint(constraintOverShoot3D, 0.0, 10**(-2))

max_iter = 10
if os.environ.get("SIMBA_SCRIPT_TEST"): # Accelerate simulation in test environment.
    max_iter = 1
    
# Run optimization algorithm
bounds = Bounds([0, 0.0, 0.0], [100,1, 1E-6],True) # 0 < Ki < 100; 0 < Kp < 1; 0 < Kd < 1E-6
tic = t.process_time()
res = differential_evolution(objective3D,
                             bounds,
                             strategy = 'rand1exp', # works well with a small population and preserve the population diversity
                             maxiter = max_iter,
                             tol = 0.01,
                             popsize = 5, 
                             init = 'latinhypercube',
                             polish = False,
                             workers = 1, # -1 to use all ressources available
                             constraints = [nonlinear_constraint_err, nonlinear_constraint_OverShoot]
               )
toc = t.process_time()
print('Elapsed time: ', toc - tic)
print ('number of simulation: ', len(simulation_history))


Elapsed time:  59.124432999999996
number of simulation:  165


In [6]:
# Print paremeters and results
optimized_parameters = res['x'].tolist()
key = create_key(optimized_parameters)
best_metrics = simulation_history.get(key)
print('Optimization Results:')
print('  Ki: ',optimized_parameters[0])
print('  Kp: ',optimized_parameters[1])
print('  Kd: ',optimized_parameters[2])
print('  risetime: ',best_metrics[0])
print('  error: ',best_metrics[1])
print('  overshoot: ',best_metrics[2])

Optimization Results:
  Ki:  70.91780316227224
  Kp:  0.004236471711019207
  Kd:  4.911388936357749e-07
  risetime:  0.0028460512635937996
  error:  0.0006930535173143015
  overshoot:  0.0021575534169395727


In [7]:
# plot
TOOLTIPS = [
    ("index", "$index"),
    ("(t, val)", "($x, $y)"),
    ]

# figure 1
p1 = figure(width = 800, height = 300, 
           title = 'Buck Converter - Output voltage',
           x_axis_label = 'time (s)', y_axis_label = 'Voltage (V)',
           active_drag='box_zoom',
           tooltips = TOOLTIPS)

#run simulation to get waveforms
pid.Ki = optimized_parameters[0]
pid.Kp = optimized_parameters[1]
pid.Kd = optimized_parameters[2]
job = buck.TransientAnalysis.NewJob()
job.Run()
Vout_signal = job.GetSignalByName('R1 - Instantaneous Voltage')
t =  np.array(Vout_signal.TimePoints)
Vout = np.array(Vout_signal.DataPoints)

p1.line(t,Vout)
output_notebook()
show(p1)

Loading BokehJS ...